---

# Chapter 19: Security Design Patterns

## Opening Context

Security is not an afterthought—it must be woven into the fabric of your application from the very first line of code. In today’s landscape of distributed systems, cloud‑native deployments, and pervasive cyber threats, a single vulnerability can lead to data breaches, financial loss, and irreparable reputational damage. Design patterns provide proven, reusable solutions to common security problems, helping you build systems that are resilient by design.

This chapter explores four categories of security design patterns:

1. **Authentication and Authorization Patterns** – How to verify identity and control access to resources.
2. **Secure Factory and Builder Patterns** – Ensuring objects are created in a valid and secure state.
3. **Security Proxy and Interceptor** – Decoupling security concerns from business logic.
4. **Input Validation and Sanitization Patterns** – Defending against injection attacks and malformed data.

By integrating these patterns into your architecture, you can enforce security consistently, reduce the risk of vulnerabilities, and create a system that is easier to audit and maintain.

---

## 19.1 Authentication and Authorization Patterns

### Intent
*Verify the identity of users or systems (authentication) and determine what actions they are permitted to perform (authorization).*

### The Problem

Every nontrivial application needs to know who is interacting with it and what they are allowed to do. Without proper authentication, attackers can impersonate legitimate users. Without proper authorization, users may access data or execute operations beyond their privileges. Implementing these mechanisms ad‑hoc leads to inconsistent, error‑prone code.

### The Solution: Standard Patterns

Authentication and authorization are often implemented using well‑established patterns and protocols:

- **Authentication**: Credential‑based (username/password), token‑based (JWT), OAuth2/OpenID Connect, API keys.
- **Authorization**: Role‑Based Access Control (RBAC), Attribute‑Based Access Control (ABAC), Policy‑Based Access Control.

#### Example: JWT Authentication Middleware

JSON Web Tokens (JWT) are a common way to handle stateless authentication. The client includes a token in the `Authorization` header; the server verifies the token and extracts user information.

```typescript
// auth/jwt.middleware.ts
import { Request, Response, NextFunction } from 'express';
import jwt from 'jsonwebtoken';

// Extend Express Request type to include user
declare global {
  namespace Express {
    interface Request {
      user?: { id: string; roles: string[] };
    }
  }
}

export const jwtMiddleware = (req: Request, res: Response, next: NextFunction) => {
  const authHeader = req.headers.authorization;
  if (!authHeader || !authHeader.startsWith('Bearer ')) {
    return res.status(401).json({ error: 'Missing or invalid token' });
  }

  const token = authHeader.substring(7); // remove 'Bearer '

  try {
    const payload = jwt.verify(token, process.env.JWT_SECRET!) as { id: string; roles: string[] };
    req.user = payload; // attach user info to request
    next();
  } catch (err) {
    return res.status(401).json({ error: 'Invalid token' });
  }
};
```

**Explanation**:
- The middleware checks for a Bearer token, verifies its signature using a secret, and attaches the decoded payload to the request object.
- Subsequent handlers can access `req.user` to know the authenticated user.

#### Example: Role‑Based Authorization

Once the user is authenticated, we need to ensure they have the required role to access certain endpoints.

```typescript
// auth/roles.guard.ts
import { Request, Response, NextFunction } from 'express';

export const requireRole = (allowedRoles: string[]) => {
  return (req: Request, res: Response, next: NextFunction) => {
    if (!req.user) {
      return res.status(401).json({ error: 'Not authenticated' });
    }

    const hasRole = req.user.roles.some(role => allowedRoles.includes(role));
    if (!hasRole) {
      return res.status(403).json({ error: 'Insufficient permissions' });
    }

    next();
  };
};
```

**Usage**:

```typescript
app.get('/admin/users', jwtMiddleware, requireRole(['admin']), (req, res) => {
  // only admins can access
});
```

#### Beyond Simple Roles: Attribute‑Based Access Control (ABAC)

For more fine‑grained control, ABAC uses attributes of the user, resource, and environment. Policies can be expressed declaratively (e.g., using a policy language like OPA).

```typescript
// Example of policy check (pseudo)
const policy = {
  "rules": [
    {
      "effect": "allow",
      "action": "read",
      "resource": "document",
      "condition": "user.department == resource.ownerDepartment"
    }
  ]
};

function checkPermission(user, action, resource): boolean {
  // Evaluate policy rules...
}
```

### Benefits

- **Separation of concerns** – Authentication and authorization are handled in dedicated modules or middleware.
- **Reusability** – The same authentication logic can protect multiple endpoints.
- **Standards compliance** – Using protocols like OAuth2 and JWT ensures interoperability.

### Pitfalls

- **Token storage** – Storing tokens securely on the client (e.g., httpOnly cookies vs. localStorage) is critical.
- **Token revocation** – JWTs are stateless; revocation requires additional mechanisms (e.g., token blacklist).
- **Over‑reliance on roles** – Roles can become too coarse; ABAC offers more flexibility but adds complexity.

---

## 19.2 Secure Factory and Builder Patterns

### Intent
*Use creational patterns to ensure that objects are constructed in a valid and secure state, preventing the creation of malformed or malicious objects.*

### The Problem

When creating complex objects, especially those that handle sensitive data or interact with external systems, it’s easy to introduce vulnerabilities. For example, a `User` object created from untrusted input might contain script tags (XSS) or SQL injection payloads. If the object is directly passed to a database or rendered in a view, it can cause security breaches.

### The Solution: Secure Factory and Builder

The **Factory Method** and **Builder** patterns can be adapted to include validation and sanitization during object creation. The factory or builder ensures that only valid, safe objects are produced, and that any required security constraints are enforced.

#### Example: Secure Factory for User Creation

A factory method that validates and sanitizes input before creating a `User` entity.

```typescript
// domain/user.ts
export class User {
  constructor(
    public readonly id: string,
    public readonly email: string,
    public readonly hashedPassword: string,
    public readonly roles: string[]
  ) {
    // Ensure invariants: email is normalized, roles are valid, etc.
    if (!this.email.includes('@')) throw new Error('Invalid email');
  }
}

// factories/user.factory.ts
import { User } from '../domain/user';
import { hashPassword } from '../utils/password';
import { sanitizeEmail, validateEmail } from '../utils/validation';

export class UserFactory {
  static async createUser(email: string, plainPassword: string, roles: string[] = ['user']): Promise<User> {
    // 1. Validate and sanitize email
    if (!validateEmail(email)) {
      throw new Error('Invalid email format');
    }
    const sanitizedEmail = sanitizeEmail(email); // e.g., normalize, trim, remove dangerous chars

    // 2. Validate password strength
    if (plainPassword.length < 8) {
      throw new Error('Password must be at least 8 characters');
    }

    // 3. Hash password (security)
    const hashedPassword = await hashPassword(plainPassword);

    // 4. Validate roles (ensure no admin escalation)
    const allowedRoles = ['user', 'manager']; // from config
    const validRoles = roles.filter(r => allowedRoles.includes(r));
    if (validRoles.length === 0) validRoles.push('user');

    // 5. Generate ID
    const id = generateUuid();

    return new User(id, sanitizedEmail, hashedPassword, validRoles);
  }
}
```

**Explanation**:
- The factory centralizes all validation and sanitization logic.
- It ensures that email is normalized, password is strong and hashed, and roles are within allowed set.
- The created `User` object is guaranteed to be in a secure, valid state.

#### Example: Secure Builder for Database Queries

A builder can construct safe database queries, protecting against SQL injection.

```typescript
// query/query.builder.ts
export class QueryBuilder {
  private table: string = '';
  private whereClauses: string[] = [];
  private parameters: any[] = [];

  from(table: string): this {
    this.table = table;
    return this;
  }

  where(column: string, operator: string, value: any): this {
    // Use parameterized placeholders
    this.whereClauses.push(`${column} ${operator} ?`);
    this.parameters.push(value);
    return this;
  }

  build(): { sql: string; params: any[] } {
    let sql = `SELECT * FROM ${this.table}`;
    if (this.whereClauses.length > 0) {
      sql += ` WHERE ${this.whereClauses.join(' AND ')}`;
    }
    return { sql, params: this.parameters };
  }
}

// Usage
const query = new QueryBuilder()
  .from('users')
  .where('email', '=', userInputEmail) // userInputEmail is treated as a parameter, not raw SQL
  .where('active', '=', true)
  .build();

// Execute with parameterized query (e.g., using mysql2)
db.execute(query.sql, query.params);
```

**Explanation**:
- The builder forces the use of parameterized queries by separating SQL structure from values.
- Even if `userInputEmail` contains malicious SQL, it will be properly escaped as a parameter, preventing injection.

### Benefits

- **Enforced invariants** – Objects are always created in a valid state.
- **Centralized security** – Validation and sanitization logic is not scattered.
- **Testability** – Factories and builders can be unit‑tested independently.

### Pitfalls

- **Over‑validation** – Too strict validation can reject legitimate input; balance security with usability.
- **Complexity** – For simple objects, a factory may be overkill; use judiciously.

---

## 19.3 Security Proxy and Interceptor

### Intent
*Control access to sensitive operations or data by inserting a security layer between the client and the real object (Proxy), or intercept calls to apply cross‑cutting security concerns (Interceptor).*

### The Problem

Security checks (authorization, audit logging, input validation) often need to be applied to many operations. Embedding them directly in business logic leads to code duplication and makes it harder to maintain security policies. Moreover, you may need to enforce different policies for different contexts (e.g., admin vs. regular user).

### The Solution: Proxy and Interceptor

- **Proxy Pattern** – A class that controls access to another object. It can perform security checks before delegating to the real object.
- **Interceptor Pattern** – A mechanism that hooks into method calls (like AOP) to apply cross‑cutting concerns. Frameworks like Spring Security or AspectJ implement this.

#### Example: Security Proxy for a Sensitive Service

Suppose we have a `DocumentService` that allows reading and writing documents. We want to enforce that users can only read documents they own (unless they are admins). We can create a proxy that checks permissions.

```typescript
// services/document.service.ts (real service)
export interface DocumentService {
  readDocument(docId: string): Promise<Document>;
  writeDocument(docId: string, content: string): Promise<void>;
}

export class RealDocumentService implements DocumentService {
  async readDocument(docId: string): Promise<Document> {
    // actual implementation
  }
  async writeDocument(docId: string, content: string): Promise<void> {
    // actual implementation
  }
}
```

```typescript
// proxies/secure-document.proxy.ts
import { DocumentService } from '../services/document.service';
import { User } from '../domain/user';

export class SecureDocumentProxy implements DocumentService {
  constructor(
    private realService: DocumentService,
    private currentUser: User
  ) {}

  async readDocument(docId: string): Promise<Document> {
    // Fetch document metadata to check ownership
    const doc = await this.realService.readDocument(docId); // but this bypasses security!
    // Better: have a separate method to get ownership info.
    // For simplicity, assume we have a document repository.
    const docMetadata = await this.getDocumentMetadata(docId);
    if (docMetadata.ownerId !== this.currentUser.id && !this.currentUser.roles.includes('admin')) {
      throw new Error('Access denied');
    }
    return this.realService.readDocument(docId);
  }

  async writeDocument(docId: string, content: string): Promise<void> {
    // Similar check for write permissions
    const docMetadata = await this.getDocumentMetadata(docId);
    if (docMetadata.ownerId !== this.currentUser.id) {
      throw new Error('Only owner can write');
    }
    await this.realService.writeDocument(docId, content);
  }

  private async getDocumentMetadata(docId: string): Promise<{ ownerId: string }> {
    // fetch from database
  }
}
```

**Explanation**:
- The proxy intercepts calls to the real service and performs authorization checks.
- The client (e.g., controller) is given the proxy instead of the real service.
- This keeps security logic separate from business logic.

#### Example: Interceptor for Method‑Level Security (Using Decorators)

In TypeScript, we can use decorators to create a simple interceptor that checks roles before executing a method.

```typescript
// decorators/authorize.decorator.ts
export function Authorize(...allowedRoles: string[]) {
  return function (target: any, propertyKey: string, descriptor: PropertyDescriptor) {
    const originalMethod = descriptor.value;

    descriptor.value = function (...args: any[]) {
      // Assume we have a way to get current user from context (e.g., using async local storage)
      const currentUser = getCurrentUser();
      if (!currentUser) throw new Error('Not authenticated');

      const hasRole = currentUser.roles.some(role => allowedRoles.includes(role));
      if (!hasRole) throw new Error('Access denied');

      return originalMethod.apply(this, args);
    };

    return descriptor;
  };
}
```

**Usage**:

```typescript
class DocumentService {
  @Authorize('admin', 'editor')
  async deleteDocument(docId: string): Promise<void> {
    // actual deletion
  }
}
```

**Explanation**:
- The decorator intercepts the method call, checks the user’s roles, and either proceeds or throws.
- This is a form of AOP (aspect‑oriented programming), common in frameworks like Spring (Java) or Castle Windsor (.NET).

### Benefits

- **Separation of concerns** – Security logic is not tangled with business logic.
- **Reusability** – The same proxy or interceptor can protect multiple services or methods.
- **Flexibility** – Security policies can be changed without modifying the core service.

### Pitfalls

- **Performance overhead** – Extra method calls and checks.
- **Complexity** – Proxies can be hard to debug if not implemented carefully.
- **Over‑proxy** – Not every method needs security; use judiciously.

---

## 19.4 Input Validation and Sanitization Patterns

### Intent
*Ensure that all input received from external sources (users, APIs, etc.) is validated for correctness and sanitized to remove or neutralize potentially dangerous content.*

### The Problem

Unvalidated input is the root cause of many security vulnerabilities: SQL injection, cross‑site scripting (XSS), command injection, path traversal, and more. Attackers can craft malicious input to exploit these weaknesses. Defending against them requires a combination of **validation** (checking that input meets expected criteria) and **sanitization** (transforming input to a safe form).

### The Solution: Validation and Sanitization Patterns

#### 1. Input Validation

Always validate input against a **whitelist** of allowed patterns, not a blacklist. Use a dedicated validator class or library.

```typescript
// validation/input.validator.ts
export class InputValidator {
  static isEmail(email: string): boolean {
    const emailRegex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    return emailRegex.test(email);
  }

  static isAlphaNumeric(str: string): boolean {
    return /^[a-zA-Z0-9]+$/.test(str);
  }

  static isValidUserId(id: string): boolean {
    // UUID format check
    const uuidRegex = /^[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}$/i;
    return uuidRegex.test(id);
  }
}
```

**Usage**:

```typescript
function createUser(email: string, username: string) {
  if (!InputValidator.isEmail(email)) {
    throw new Error('Invalid email');
  }
  if (!InputValidator.isAlphaNumeric(username)) {
    throw new Error('Username must be alphanumeric');
  }
  // proceed
}
```

#### 2. Sanitization

Sanitization transforms input to make it safe for its intended use. For example:
- **HTML/JS sanitization** to prevent XSS (using libraries like DOMPurify).
- **SQL parameterization** (not exactly sanitization, but the best defence).
- **Shell argument escaping**.

```typescript
// sanitization/html.sanitizer.ts
import DOMPurify from 'dompurify';
import { JSDOM } from 'jsdom';

const window = new JSDOM('').window;
const purify = DOMPurify(window);

export function sanitizeHtml(unsafeHtml: string): string {
  return purify.sanitize(unsafeHtml, { ALLOWED_TAGS: ['b', 'i', 'em', 'strong'] });
}
```

**Usage**:

```typescript
const userComment = req.body.comment; // could contain <script>alert('xss')</script>
const safeComment = sanitizeHtml(userComment);
// store safeComment in database
```

#### 3. Validation and Sanitization Pipeline

Combine validation and sanitization in a pipeline for complex input.

```typescript
// pipeline/input.pipeline.ts
interface ValidationRule<T> {
  validate(input: T): boolean;
  sanitize?(input: T): T;
}

class InputPipeline<T> {
  private rules: ValidationRule<T>[] = [];

  addRule(rule: ValidationRule<T>): this {
    this.rules.push(rule);
    return this;
  }

  process(input: T): T {
    let processed = input;
    for (const rule of this.rules) {
      if (rule.sanitize) {
        processed = rule.sanitize(processed);
      }
      if (!rule.validate(processed)) {
        throw new Error('Validation failed');
      }
    }
    return processed;
  }
}

// Example rule: email
const emailRule: ValidationRule<string> = {
  validate: InputValidator.isEmail,
  sanitize: (email) => email.trim().toLowerCase()
};

// Usage
const pipeline = new InputPipeline<string>().addRule(emailRule);
const safeEmail = pipeline.process('  USER@Example.com  '); // 'user@example.com'
```

#### 4. Output Encoding (Prevention of XSS)

When displaying user‑supplied data in HTML, encode it to prevent script execution.

```typescript
// utils/encoding.ts
export function encodeHtml(str: string): string {
  return str
    .replace(/&/g, '&amp;')
    .replace(/</g, '&lt;')
    .replace(/>/g, '&gt;')
    .replace(/"/g, '&quot;')
    .replace(/'/g, '&#039;');
}
```

**Usage** (in a template engine):

```typescript
const unsafeString = '<script>alert(1)</script>';
const safeString = encodeHtml(unsafeString); // '&lt;script&gt;alert(1)&lt;/script&gt;'
```

### OWASP Guidelines

The Open Web Application Security Project (OWASP) provides comprehensive guidance. Key principles:
- **Validate all input** – on server side, even if client‑side validation exists.
- **Use parameterized queries** – never concatenate SQL.
- **Encode output** – based on context (HTML, JavaScript, CSS, URL).
- **Use secure libraries** – for validation, sanitization, and encoding.

### Benefits

- **Defense in depth** – Multiple layers protect against injection.
- **Centralized logic** – Validation and sanitization are in one place, making it easier to audit.
- **Reduced attack surface** – Malformed input is rejected early.

### Pitfalls

- **Over‑sanitization** – Removing legitimate characters can break functionality.
- **Validation bypass** – Ensure validation is done on the server; client‑side is for UX only.
- **Character set issues** – Consider Unicode normalization and encoding.

---

## Chapter Summary

Security design patterns are essential for building robust, resilient systems. This chapter covered four key areas:

1. **Authentication and Authorization** – Using middleware and guards to verify identity and enforce permissions. Patterns like JWT and RBAC provide scalable, standardised solutions.

2. **Secure Factory and Builder** – Ensuring objects are created in a valid, secure state by centralising validation, sanitization, and secure defaults. This prevents many vulnerabilities at the point of object creation.

3. **Security Proxy and Interceptor** – Decoupling security concerns from business logic using proxies or interceptors. These patterns allow you to apply access control, logging, and validation without cluttering core code.

4. **Input Validation and Sanitization** – Never trust user input. Validate against whitelists, sanitize dangerous content, and encode output. These practices form the foundation of defence against injection attacks.

**Key Insight**: Security is not a feature; it's a property of the entire system. By embedding these patterns into your architecture, you create a system that is secure by design, easier to maintain, and more resistant to attacks.

---

## Next Chapter Preview

**Chapter 20: Domain-Driven Design (DDD) (Entities, Value Objects, Aggregates, Repositories, Bounded Contexts)**

We now turn to Domain‑Driven Design, a set of patterns for modelling complex business domains. Chapter 20 will explore core tactical patterns: **Entities** and **Value Objects** for modelling domain concepts, **Aggregates** for consistency boundaries, **Repositories** for data access, and **Bounded Contexts** for dividing large models. These patterns, combined with the architectural styles we've covered, will help you build systems that truly reflect the business they serve.

